In [103]:
import os

import numpy as np
from scipy.ndimage import zoom
import cv2
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.applications import EfficientNetB0,EfficientNetB1
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate, BatchNormalization
from tf_explain.core.grad_cam import GradCAM
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, CSVLogger, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
from keras import backend as K


In [124]:
class CNNDataGenerator(Sequence):
    def __init__(self, datapath, metadata, shape=(256, 256), batch_size=1, n_classes=5, shuffle=True, **kwargs):
      super().__init__(**kwargs)
      self.datapath = datapath
      self.metadata = metadata
      self.shape = shape
      self.batch_size = batch_size
      self.indices = self.metadata.index.tolist()
      self.n_classes = n_classes
      self.shuffle = shuffle
      self.on_epoch_end()
      
    def __len__(self):
      return (len(self.indices) // self.batch_size)
    
    def __getitem__(self, index):
      indexes = self.index[index*self.batch_size : (index+1)*self.batch_size]
      batch = [self.indices[k] for k in indexes]
      
      X,y = self.__get_data(batch)
      
      print(X.shape, y.shape)
      return (X),y
    
    def on_epoch_end(self):
      self.index = np.arange(len(self.indices))
      if self.shuffle:
        np.random.shuffle(self.index)
    
    def __get_data(self, batch):
      X_sinogram = np.zeros((len(batch), *self.shape), dtype='float32')
      y = np.zeros((len(batch), self.n_classes), dtype='float32')
      
      for i, id in enumerate(batch):
        try:
          sinogram_image = np.load(self.datapath + '/' + self.metadata.loc[id, 'Image'] + '.npy')
          a = sinogram_image[:,:,0]
          resized_image = cv2.resize(a, self.shape)
          X_sinogram[i,:,:] = resized_image
          y[i, :] = self.metadata.loc[id, ['any']].to_numpy(dtype='float32')
        except Exception as e:
          print(f"Error loading image with image_id': {self.metadata.loc[id, 'Image']}, Error: {str(e)}")
          
      return X_sinogram, y
      

In [125]:
input_shape = (256, 256)

In [126]:
def custom_accuracy(threshold=0.5):
    def thresholded_accuracy(y_true, y_pred):
        thresholded_pred = tf.cast(tf.greater(y_pred, threshold), tf.float32)
        return tf.keras.metrics.binary_accuracy(y_true, thresholded_pred)
    return thresholded_accuracy

In [127]:
def create_and_compile_model(input_shape, learning_rate=0.001):
    image_input = Input(shape=input_shape, name='image_input')
    image_backbone = EfficientNetB0(weights=None, include_top=False, input_tensor=image_input, input_shape=input_shape)
    
    for layer in image_backbone.layers:
        layer._name = 'Image_'+layer.name
        
    image_features = GlobalAveragePooling2D()(image_backbone.output)
    
    # additional layer
    x = Dense(256, activation='relu')(image_features)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    
    output = Dense(1, activation='sigmoid')(x) # Binary Classification
    
    # final model
    model = tf.keras.Model(inputs=[image_input], outputs=output)
    
    # compile the model with appropriate loss, optimizer, and learning rate
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    
    model.compile(optimizer=optimizer, 
                  loss='binary_crossentropy',
                  metrics=[
                        tf.keras.metrics.BinaryAccuracy(),
                        tf.keras.metrics.Precision(),
                        tf.keras.metrics.Recall(),
                        # F1Score(),
                        tf.keras.metrics.AUC(curve='ROC'),
                        tf.keras.metrics.AUC(curve='PR')
                    ]
                  )
    
    return model

In [128]:
model = create_and_compile_model(input_shape=(*input_shape, 1), learning_rate=0.001)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 256, 256,  │          0 │ image_input[0][0] │
│ (Rescaling)         │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_1     │ (None, 256, 256,  │          3 │ rescaling_1[0][0] │
│ (Normalization)     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 257, 257,  │          0 │ normalization_1[… │
│ (ZeroPadding2D)     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 128, 128,  │        288 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 128, 128,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 128, 128,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 128, 128,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 128, 128,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 128, 128,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 128, 128,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 128, 128,  │        512 │ block1a_se_excit… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 128, 128,  │         64 │ block1a_project_

 Total params: 4,410,464 (16.82 MB)

 Trainable params: 4,368,189 (16.66 MB)

 Non-trainable params: 42,275 (165.14 KB)

In [ ]:
out_path = '/workspace/ich-classification/out'
def train_model(train_dataset, valid_dataset, initial_learning_rate=0.001, epochs=20):
    input_shape = (256, 256, 1)
    model = create_and_compile_model(input_shape, learning_rate=initial_learning_rate)
    
    checkpoint_path = os.path.join(out_path, 'checkpoints')
    tensorboard_log_dir = os.path.join(out_path, "logs")
    csv_filename = os.path.join(out_path, "training_metrics.csv")
    
    os.makedirs(checkpoint_path, exist_ok=True)
    os.makedirs(tensorboard_log_dir, exist_ok=True)
    
    # Save checkpoints with timestamps and epoch numbers
    checkpoint_callback = ModelCheckpoint(
        os.path.join(checkpoint_path, 'best_model.weights.h5'),
        monitor='val_loss',
        save_best_only=True,  # Save all weights
        save_weights_only=True,
        verbose=1,
        mode='min'
    )
    
    tensorboard_callback = TensorBoard(
        log_dir=tensorboard_log_dir,
        histogram_freq=1,  # How often to compute histograms (set to 1 for every epoch)
        write_images=True,  # Save model architecture as an image
        write_graph=True  # Write the computation graph to a file
    )
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', verbose=1)
    
    # Create a CSVLogger to log training metrics to a CSV file
    csv_logger = CSVLogger(csv_filename, separator=',', append=True)
    
    callbacks = [checkpoint_callback, csv_logger, early_stopping,tensorboard_callback]
    
    # Train the model for a specified number of epochs with a specified number of steps per epoch
    history = model.fit(train_dataset, validation_data=valid_dataset, 
                        epochs=epochs, #steps_per_epoch=steps_per_epoch, validation_steps = validation_steps, 
                        callbacks=callbacks
                        )
    
        # Save the trained model
    model.save(os.path.join(out_path, "final_model.h5"))
    
    # Save training history to a text file
    with open(os.path.join(out_path, "training_history.txt"), "w") as history_file:
        history_file.write(str(history.history))

In [135]:
# train_dataset, valid_dataset, initial_learning_rate=0.001, epochs=20
def test_model(test_dataset, initial_learning_rate, weights_path=None):
    # Define input shape for each modality
    input_shape = (256, 256, 1)
    
    # Create and compile the model
    model = create_and_compile_model(input_shape,learning_rate=initial_learning_rate)
    
    # Load pre-trained weights if provided
    if weights_path:
        model.load_weights(weights_path)
        # model.load_weights(os.path.join(args.output_dir, "final_model.h5"))
        print("Loaded the weights successfully")
    
    # Evaluate the model on the test dataset
    test_loss, test_accuracy, test_auc, test_sensitivity, test_specificity = model.evaluate(test_dataset)
    print(f"Test Loss: {test_loss}")
    print(f"Test Accuracy: {test_accuracy}")
    print(f"Test AUC: {test_auc}")
    print(f"Test Sensitivity: {test_sensitivity}")
    print(f"Test Specificity: {test_specificity}")
    
    # Create a dictionary to store the test results
    test_results = {
        "Test Loss": [test_loss],
        "Test Accuracy": [test_accuracy],
        "Test AUC": [test_auc],
        "Test Sensitivity": [test_sensitivity],
        "Test Specificity": [test_specificity]
    }
    
    # Convert the dictionary into a Pandas DataFrame
    test_results_df = pd.DataFrame(test_results)
    
    # Specify the path for the CSV file
    result_csv_path = os.path.join(out_path, "test_results.csv")
    
    # Save the DataFrame to the CSV file
    test_results_df.to_csv(result_csv_path, index=False)

In [136]:
def set_visible_gpus(gpu_devices):
    # Set the CUDA_VISIBLE_DEVICES environment variable to the selected GPUs
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu_devices

In [137]:
def main(mode='train'):
    K.clear_session()
    metadata_dir = '/workspace/datasets/train-series-splitted-metadata'
    set_visible_gpus('0')

    physical_devices = tf.config.experimental.list_physical_devices('GPU')
    if len(physical_devices) == 0:
        print("No GPUs available. Please check your GPU configuration.")
        return
    
    # Create a MirroredStrategy with all available GPUs
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:" + str(i) for i in range(len(physical_devices))])
    
    with strategy.scope():
        train = pd.read_csv(os.path.join(metadata_dir, "train800.csv"))
        valid = pd.read_csv(os.path.join(metadata_dir, "valid100.csv"))
        test_yn = pd.read_csv(os.path.join(metadata_dir, "test100.csv"))
        
        data_path = '/workspace/datasets/processed/sinograms'
        train_dataset = CNNDataGenerator(datapath=data_path, metadata=train, shape=(256, 256), batch_size=64, n_classes=6, shuffle=True)
        valid_dataset = CNNDataGenerator(datapath=data_path, metadata=valid, shape=(256, 256), batch_size=64, n_classes=6, shuffle=True)
        test_dataset = CNNDataGenerator(datapath=data_path, metadata=test_yn, shape=(256, 256), batch_size=64, n_classes=6, shuffle=False)
    
        print(train_dataset.shape)
        if mode == "train":
            train_model(train_dataset, valid_dataset, initial_learning_rate=0.001, epochs=20)
        else:
            test_model(test_dataset, initial_learning_rate=0.001, weights_path=None)

In [138]:
main('train')

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
(256, 256)


ValueError: `input_shape` must be a tuple of three integers.